In [2]:
## Importing necessary libraries
import os
import pandas as pd
import numpy as np
from glob import glob

In [3]:
## Combining all 12 Data sets into one data frame

files = glob('2021*.csv') #creating an object containing the list of files

data = pd.concat((pd.read_csv(file) for file in files), ignore_index=True)

In [4]:
data.shape

(5595063, 13)

In [5]:
## Dropping ride_id, start_station, end_station, start_station_id, end_station_id
### for the purposes of this analysis we will not be needing these columns

col_to_drop = ['ride_id','start_station_name','start_station_id','end_station_name','end_station_id']

data_2 = data.drop(col_to_drop, axis=1)

In [6]:
data_2.rename(columns={'rideable_type':'Bike_type','member_casual':'member_type'},inplace=True) #changing names of columns to make more sense

In [7]:
data_2.fillna(0,inplace=True) #replacing null lat and long values with zero to make calculations valid later on

In [8]:
## Sub Question 1: Between casuals and members, how long do rides usually last?

#creating new column to calculate travel time
data_2['travel_time'] = pd.to_datetime(data_2['ended_at']) - pd.to_datetime(data_2['started_at'])

In [9]:
data_2

,Bike_type,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,member_type,travel_time
0,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,41.900341,-87.696743,41.890000,-87.720000,member,0 days 00:10:25
1,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,41.900333,-87.696707,41.900000,-87.690000,member,0 days 00:04:04
2,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,41.900313,-87.696643,41.900000,-87.700000,member,0 days 00:01:20
3,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,41.900399,-87.696662,41.920000,-87.690000,member,0 days 00:11:42
4,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,41.900326,-87.696697,41.900000,-87.700000,casual,0 days 00:00:43
...,...,...,...,...,...,...,...,...,...
5595058,electric_bike,2021-12-12 13:36:55,2021-12-12 13:56:08,41.882289,-87.639752,41.890000,-87.610000,casual,0 days 00:19:13
5595059,electric_bike,2021-12-06 19:37:50,2021-12-06 19:44:51,41.882123,-87.640053,41.889106,-87.638862,member,0 days 00:07:01
5595060,electric_bike,2021-12-02 08:57:04,2021-12-02 09:05:21,41.881956,-87.639955,41.880254,-87.629603,member,0 days 00:08:17
5595061,electric_bike,2021-12-13 09:00:26,2021-12-13 09:14:39,41.860000,-87.720000,41.850000,-87.710000,member,0 days 00:14:13


In [10]:
## Sub Question 2: How far did the casual travel in comparison to members?

#create variables for lat and long

lat1 = (data_2['start_lat'])
lat2 = (data_2['end_lat'])
lon1 = (data_2['start_lng'])
lon2 = (data_2['end_lng'])

data_2['dlat'] = (lat2 - lat1)**2 #delta lat squared
data_2['dlon'] = (lon2 - lon1)**2 #delta lon squared


#calculate distance between two points assuming bike travel distance is less than 1000 miles, we can ignore curvature of earth
data_2['travel_distance_notsquare'] = data_2['dlat'] + data_2['dlon']

data_2['travel_distance_final'] = data_2['travel_distance_notsquare']**(1/2)

In [11]:
#creating new data frame in order to drop unneeded columns (dlat,dlon,travel_distance_notsquare)
data_3 = data_2.drop(['dlat','dlon','travel_distance_notsquare'],axis= 1)

In [12]:
## Going back to Sub Question 1, adding new column converting travel time into minutes

data_3['final_travel_time'] = data_3['travel_time'] / np.timedelta64(1,'m')

In [13]:
data_3

,Bike_type,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,member_type,travel_time,travel_distance_final,final_travel_time
0,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,41.900341,-87.696743,41.890000,-87.720000,member,0 days 00:10:25,0.025452,10.416667
1,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,41.900333,-87.696707,41.900000,-87.690000,member,0 days 00:04:04,0.006715,4.066667
2,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,41.900313,-87.696643,41.900000,-87.700000,member,0 days 00:01:20,0.003372,1.333333
3,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,41.900399,-87.696662,41.920000,-87.690000,member,0 days 00:11:42,0.020703,11.700000
4,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,41.900326,-87.696697,41.900000,-87.700000,casual,0 days 00:00:43,0.003319,0.716667
...,...,...,...,...,...,...,...,...,...,...,...
5595058,electric_bike,2021-12-12 13:36:55,2021-12-12 13:56:08,41.882289,-87.639752,41.890000,-87.610000,casual,0 days 00:19:13,0.030735,19.216667
5595059,electric_bike,2021-12-06 19:37:50,2021-12-06 19:44:51,41.882123,-87.640053,41.889106,-87.638862,member,0 days 00:07:01,0.007084,7.016667
5595060,electric_bike,2021-12-02 08:57:04,2021-12-02 09:05:21,41.881956,-87.639955,41.880254,-87.629603,member,0 days 00:08:17,0.010491,8.283333
5595061,electric_bike,2021-12-13 09:00:26,2021-12-13 09:14:39,41.860000,-87.720000,41.850000,-87.710000,member,0 days 00:14:13,0.014142,14.216667


In [21]:
## Exporting Data_3 into a csv to do further analysis and visualization in Tableau

# data_3.to_csv('final_divvy_tripdata_20220829.csv') 

In [15]:
data_3.groupby('member_type', as_index=False)['final_travel_time'].mean()

,member_type,final_travel_time
0,casual,32.000947
1,member,13.632841


In [17]:
data_3.groupby('member_type', as_index=False)['travel_distance_final'].mean()

,member_type,travel_distance_final
0,casual,0.156891
1,member,0.062212


In [19]:
# Index 0 = Casual Riders
# Index 1 = Member Riders and so on and so forth

data_3.groupby(['Bike_type', 'member_type'], as_index=False)['member_type'].count() 

,Bike_type,member_type
0,classic_bike,1266657
1,classic_bike,1984371
2,docked_bike,312342
3,docked_bike,1
4,electric_bike,950006
5,electric_bike,1081686
